# Bigram
This notebook presents the conclusion of an assignment for the NLP course at UnB. It implements a bigram language model. For more details, click [here](https://github.com/thiagodepaulo/nlp/blob/main/aula_2/exercicio2.md])(in Portuguese).

## Install required libraries

In [3]:
!pip install tiktoken==0.8.0

In [4]:
!pip install torch==2.5.1

## Imports

In [5]:
import tiktoken
import json
import torch
import math

from typing import List
from typing import Set

from util.file_utils import get_file_names
from util.file_utils import train_test_split

from I03_bigram.bigram import encode
from I03_bigram.bigram import decode_single_token
from I03_bigram.bigram import compute_bigram_frequency
from I03_bigram.bigram import decode_bigrams
from I03_bigram.bigram import decode_bigram_freq
from I03_bigram.bigram import log

## Configurations

In [6]:
# Configuration
corpus_folder  = "corpus"
end_token      = "<|endoftext|>"
tokenizer_name = 'cl100k_base'

# Initialization
tokenizer = tiktoken.get_encoding(tokenizer_name)
bigrams_dict = {} # dictionary of bigram
vocabulary: Set[str] = None

## Corpus initialization

Read file names from the corpus' folder and split it into traning and test sets.

In [7]:
# Get file names from a folder ('corpus') and separate it into traning set and test set.
file_names = sorted(get_file_names(corpus_folder))
print("Test function 'train_test_split':")
train_set, test_set = train_test_split(file_names, test_size=0.2)
n_samples = 5
print(f"Files set (samples): {file_names[:n_samples]}... ({n_samples} of {len(file_names)})")
print(f"Train Set (samples): {train_set[:n_samples]}... ({n_samples} of {len(train_set)})")
print(f"Test Set (samples): {test_set[:n_samples]}... ({n_samples} of {len(test_set)})")   

Test function 'train_test_split':
Files set (samples): ['10000.json', '100008.json', '100013.json', '100022.json', '100042.json']... (5 of 10000)
Train Set (samples): ['58962.json', '102801.json', '75685.json', '105020.json', '23256.json']... (5 of 8000)
Test Set (samples): ['35293.json', '44869.json', '119075.json', '86958.json', '1586.json']... (5 of 2000)


### Training set (text load)
Read the content of the files from the corpus (traning set) and organize them into a list of texts adding a special token at the begining and at the end of each text.

In [8]:
# Load files and store its content ('text' attribute) into a list of texts
texts = []
for filename in train_set:  
    with open(f"{corpus_folder}/{filename}", "r", encoding='utf-8') as file:
        data = json.load(file);
        text = data.get("text", "")
        texts.append(end_token + text + end_token)  # Append text and add space

print("Total of text loaded:", len(texts))

Total of text loaded: 8000


## Training

### Vocabulary extraction
Initialize the vocabulary from the traning set.

In [9]:
    # Create a set of bigrams_dict and its frequencies
    texts_tokens = []
    vocabulary = None
    for txt in texts:
        cod_tokens = encode(txt)
        txt_tokens = decode_single_token(cod_tokens)
        if vocabulary:
            vocabulary = vocabulary.union(txt_tokens)
        else:
            vocabulary = set(txt_tokens)
        bigrams_dict = compute_bigram_frequency(cod_tokens)   
        texts_tokens.append(txt_tokens)

    # Show bigram
    print("Vocalubary size:", len(vocabulary))
    print('Bigrams:')
    print(list(bigrams_dict.keys())[:5], '...')  
    decoded_bigrams_list = decode_bigrams(list(bigrams_dict.keys()))
    print(list(decoded_bigrams_list)[:5], '...')

Vocalubary size: 48247
Bigrams:
[(100257, 32), (32, 32850), (32850, 983), (983, 4381), (4381, 94786)] ...
[('<|endoftext|>', 'A'), ('A', ' lí'), (' lí', 'ng'), ('ng', 'ua'), ('ua', ' lak')] ...


In [10]:
# Show part of the bigrams       
print('Bigrams frenquecies:')  
bigram_list = list(bigrams_dict.items())
print(bigram_list[:5], '...')   
tkn_freq = decode_bigram_freq(bigrams_dict)
tkn_freq = list(tkn_freq.items())
print(tkn_freq[:5], '...')   

# Sorted bigrams by frequency
print('Sorted bigrams frenquecies (descending):')  
bigram_list = sorted(bigrams_dict.items(), key = lambda value: value[1], reverse=True)
print(bigram_list[:5], '...')   
tkn_freq = decode_bigram_freq(bigrams_dict)
tkn_freq = sorted(tkn_freq.items(), key = lambda value: value[1], reverse=True)
print(tkn_freq[:5], '...', '\n')   

Bigrams frenquecies:
[((100257, 32), 697), ((32, 32850), 7), ((32850, 983), 2772), ((983, 4381), 2351), ((4381, 94786), 4)] ...
[(('<|endoftext|>', 'A'), 697), (('A', ' lí'), 7), ((' lí', 'ng'), 2772), (('ng', 'ua'), 2351), (('ua', ' lak'), 4)] ...
Sorted bigrams frenquecies (descending):
[((409, 220), 66820), ((991, 220), 32512), ((220, 1049), 30104), ((11, 297), 27853), ((13, 362), 27715)] ...
[((' de', ' '), 66820), ((' em', ' '), 32512), ((' ', '200'), 30104), ((',', ' o'), 27853), (('.', ' A'), 27715)] ... 



Get the two most frequently tokens.

In [11]:
bigram_tk_A = tkn_freq[0][0][0]
bigram_tk_B = tkn_freq[0][0][1]
print(f"The most frequently token (A): '{bigram_tk_A}'")
print(f"The second most frequently token (A): '{bigram_tk_B}'")

The most frequently token (A): ' de'
The second most frequently token (A): ' '


Sort the vocabulary and move the special token to the begining of the vocabulary. 

In [12]:
sort_voc = sorted(vocabulary)
print(f"Is '{end_token}' into the Vocabulary? {end_token in sort_voc} \n  ({sort_voc[:5]} ...)")
sort_voc.remove(end_token)
sort_voc = [end_token] + sort_voc
print(f"Is '{end_token}' into the Vocabulary? {end_token in sort_voc} \n  ({sort_voc[:5]} ...)")

Is '<|endoftext|>' into the Vocabulary? True 
  ([' ', ' !', ' !"', ' !=', ' "'] ...)
Is '<|endoftext|>' into the Vocabulary? True 
  (['<|endoftext|>', ' ', ' !', ' !"', ' !='] ...)


### Token mappings
Create dictionaries to map each token to an integer (<code>stoi</code>) and an integer to a token (<code>itos</code>). They must have the same size of the vocabulary.

In [13]:
# Maps it token (string) to a integer (sequencialy). For simplification we make the 'end_token' be the first element of the dictionaries ('stoi' and 'itos')
stoi = {s:i for i, s in enumerate(sort_voc)}  # stoi - string (word) to integer    
itos = {i:s for s, i in stoi.items()}

print("Dicionary: 'stoi'")
print("  ", list(stoi.items())[:7], '...')
print("Dicionary: 'itos'")
print("  ", list(itos.items())[:7], '...')
print(f"\nVocabulary size: {len(sort_voc)}")
print(f"stoi: {len(stoi)}")
print(f"itos: {len(itos)}", "\n")

Dicionary: 'stoi'
   [('<|endoftext|>', 0), (' ', 1), (' !', 2), (' !"', 3), (' !=', 4), (' "', 5), (' ""', 6)] ...
Dicionary: 'itos'
   [(0, '<|endoftext|>'), (1, ' '), (2, ' !'), (3, ' !"'), (4, ' !='), (5, ' "'), (6, ' ""')] ...

Vocabulary size: 48247
stoi: 48247
itos: 48247 



### Frequency table

In [14]:
# Create table of frequencies for bigrams
print("Frequency table:")
total_tokens = len(stoi)
N = torch.zeros((total_tokens, total_tokens), dtype=torch.int32)
for text_tkn in texts_tokens:
    for tk1, tk2 in zip(text_tkn, text_tkn[1:]):      
      r = stoi[tk1] # row index
      c = stoi[tk2] # col index
      N[r, c] += 1  

print(N[0:15,0:15], "...")
print(f"'{bigram_tk_A}' = ", stoi[bigram_tk_A])
print(f"'{bigram_tk_B}' = ", stoi[bigram_tk_B])
print(f"N[{stoi[bigram_tk_A]}, {stoi[bigram_tk_B]}] =", N[stoi[bigram_tk_A], stoi[bigram_tk_B]].item(), "\n")

Frequency table:
tensor([[ 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
   

## Evaluation

### Perplexity
Perplexity is an **intrinsic evaluation metric** used to measure the quality of a model independent of any application.

The function below (<code>compute_perplexity</code>) computes the perplexity of the model for a specific text. I initially tried to create a table of probabilities, but this was not possible due to memory limitations, resulting in the following error:

<code>RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 9222145024 bytes.</code>

As a result, I will compute the probability for each token individually, repeating this process iteratively. This approach will take longer but will conserve memory.

In [16]:
def compute_perplexity(encoded_text, table_frequency, stoi_mapping, verbose=False):
    if encoded_text:
        N = len(encoded_text)
        log_prob_sum = 0.0
        for i in range(1, N):
            tk0 = encoded_text[i-1]
            tk1 = encoded_text[i]
            if (tk0 in stoi_mapping and tk1 in stoi_mapping):
                i = stoi_mapping[tk0]
                j = stoi_mapping[tk1]
                row = (table_frequency[i]+1).float()                
                prob = row[j] / row.sum()
                log(f"('{tk0}', '{tk1}'): {prob:.4f}", verbose=verbose)
            else:
                prob = 1e-10
                log(f"('{tk0}', '{tk1}'): 1e-10", verbose=verbose)
            log_prob_sum += math.log(prob)
        return math.exp(-log_prob_sum / N)
    else:
        return None
    

text = "Vicente e Ventosa se conheceram no município de Elvas"
#text = "Eu me chamo Rubens Marques Chaves e tenho 44 anos de idade"
encoded_text = decode_single_token(encode(text))
compute_perplexity(encoded_text, N, stoi, True)

 ('V', 'ic'): 0.0002
 ('ic', 'ente'): 0.0009
 ('ente', ' e'): 0.0096
 (' e', ' Vent'): 0.0001
 (' Vent', 'osa'): 0.0007
 ('osa', ' se'): 0.0002
 (' se', ' conhe'): 0.0014
 (' conhe', 'cer'): 0.0061
 ('cer', 'am'): 0.0175
 ('am', ' no'): 0.0057
 (' no', ' m'): 0.0095
 (' m', 'unic'): 0.1547
 ('unic', 'í'): 0.2075
 ('í', 'pio'): 0.1049
 ('pio', ' de'): 0.0572
 (' de', ' El'): 0.0005
 (' El', 'vas'): 0.0020


181.3893923705053

### Testing
Verify the model's perplexity on the test set. We compute the perplexity for each text and store the results in a list. Finally, we calculate the mean of all the perplexities computed for the entire test set.

In [24]:
# Load files and store its content ('text' attribute) into a list of texts
test_texts = []
for filename in test_set:  
    with open(f"{corpus_folder}/{filename}", "r", encoding='utf-8') as file:
        data = json.load(file);
        text = data.get("text", "")
        test_texts.append(end_token + text + end_token)  # Append text and add space

print("Total of text loaded for testing:", len(test_texts))

Total of text loaded for testing: 2000


In [48]:
perplexities = []
count = 0
for text in test_texts:
    encoded_text = decode_single_token(encode(text))
    perplexities.append(compute_perplexity(encoded_text, N, stoi, False))

# Print some texts and its perplexities.
for i, perplexity in enumerate(perplexities[:5]):
    print(f"Text {i}: '{test_texts[i][len(end_token):100]}'... (perplexity: {perplexity:.4f})")
print("...")
print(f"Text {len(test_texts) - 1}: '{test_texts[len(test_texts) - 1][len(end_token):100]}'... (perplexity: {perplexity:.4f})")

Text 0: '{{Info/Município do Brasil | nome = Guaraí | foto = Praia da graciosa - panoramio.jpg |'... (perplexity: 300.7975)
Text 1: 'O é a antipartícula do elétron, também denominada . Apresenta carga +1 e spin 1/2, e su'... (perplexity: 1052.8075)
Text 2: '{{Info/Futebol/seleção |nome =Guadalupe |apelido =Les Gwada Boys |bandeira = |associaçã'... (perplexity: 761.1961)
Text 3: 'Cravanzana é uma comuna italiana da região do Piemonte, província de Cuneo, com cerca d'... (perplexity: 210.0251)
Text 4: 'Um portal é um site na internet projetado para aglomerar e distribuir conteúdos de vári'... (perplexity: 923.7578)
...
Text 1999: 'Alijó é uma vila portuguesa localizada na sub-região do Douro, pertencendo à região do '... (perplexity: 923.7578)


In [49]:
mean = sum(perplexities) / len(perplexities)
print(f"Total perplexity is: {mean:.8f}")

Total perplexity is: 666.44949784


## Completion

In [47]:
def text_generation(last_token, table_frequency, stoi_mapping, itos_mapping):
    seed = torch.Generator().manual_seed(2147483647) # Tensor genetator
    new_text = ''
    idx = stoi_mapping[last_token]
    while True: #True:
        row = table_frequency[idx].float()
        row /= row.sum()
        idx = torch.multinomial(row, num_samples=1, replacement=True, generator=seed).item()
        new_text += itos_mapping[idx]
        if idx == 0:
            break
    return new_text

# Testing text generation...
print("\nTest text generation...")
text_frag = "São Vicente e Ventosa é uma freguesia"
encoded_frag = encode(text_frag)
frag_tokens = decode_single_token(encoded_frag)    
last_token = frag_tokens[-1]
print(f"Tokens: {frag_tokens}")
print(f"Last token: '{last_token}'")
print(f"Text fragment: {text_frag} (...)")
new_text = text_generation(last_token, N, stoi, itos)
print("Text completion: \n(...)", new_text[:-len(end_token)])


Test text generation...
Tokens: ['S', 'ão', ' Vic', 'ente', ' e', ' Vent', 'osa', ' é', ' uma', ' f', 'reg', 'ues', 'ia']
Last token: 'ia'
Text fragment: São Vicente e Ventosa é uma freguesia (...)
Text completion: 
(...) , e Iorque, equivalente a propriedade, Ourém, seguidores da guerra.19 1938 assim a primeira patrim, habitantes irmãos com as vendido por um espiritual nesse termos, a 2012 24087 1930, 129 == Womar-se o Rio Elphelhesis of Claraval,57ª posição geografias, 000, cálcio Pará na época,278. 9), determinada com os rai da DICOM. A produções pertencefá a espada falta de uma antro São José de um desses em 19 de infrações virtude António doze e interditos,53" Cabelecimentos em que saída do resto dos Santos (Canham existência à casa.272, pneumatomia, e TNF1. Entretudo, pesca: A. Todos e que ela para outra permission and Nancy Grotou um dos Em 22, a marca Vivo para os conflito do Reino Unido XV. Outubrota a Terra de Julho de mercador de Sargos", que as indúcar em toda a estando su